## 影片辨識


In [4]:
import torch
print(torch.cuda.is_available())  # 確保為 True
print(torch.cuda.device_count())  # 確保大於 0
print(torch.cuda.get_device_name(0))  # 顯示 GPU 名稱


True
1
NVIDIA GeForce RTX 3050 Laptop GPU


In [5]:
import os
HOME = os.getcwd()
print (HOME)

C:\Users\Admin\Desktop\yolov9


In [6]:
%cd {HOME}/yolov9

C:\Users\Admin\Desktop\yolov9\yolov9


In [3]:
import cv2
import os
import subprocess
import shutil
from pathlib import Path
import logging
from typing import Optional
import re  

class VideoProcessor:
    def __init__(self, video_path: str, frames_folder: str, detected_folder: str, 
                 output_video: str, weights_path: str, target_fps: int = 5,
                 sample_rate: float = 1.0):
        self.video_path = Path(video_path)
        self.frames_folder = Path(frames_folder)
        self.detected_folder = Path(detected_folder)
        self.output_video = Path(output_video)
        self.weights_path = Path(weights_path)
        self.target_fps = target_fps
        self.sample_rate = sample_rate
        
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)
        
    def setup_folders(self):
        """建立必要的資料夾"""
        try:
            self.frames_folder.mkdir(parents=True, exist_ok=True)
            self.detected_folder.mkdir(parents=True, exist_ok=True)
            self.output_video.parent.mkdir(parents=True, exist_ok=True)
        except Exception as e:
            self.logger.error(f"建立資料夾時發生錯誤: {e}")
            raise

    def extract_frames(self) -> tuple[int, int, int]:
        """從影片中提取幀"""
        if not self.video_path.exists():
            raise FileNotFoundError(f"找不到影片檔案: {self.video_path}")

        cap = cv2.VideoCapture(str(self.video_path))
        if not cap.isOpened():
            raise RuntimeError("無法開啟影片檔案")

        try:
            frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            source_fps = int(cap.get(cv2.CAP_PROP_FPS))
            
            # 計算需要跳過的幀數
            frames_to_skip = int(source_fps / self.sample_rate)
            if frames_to_skip < 1:
                frames_to_skip = 1
                
            self.logger.info(f"來源影片 FPS: {source_fps}")
            self.logger.info(f"取樣率: 每 {1/self.sample_rate:.2f} 秒一幀")
            self.logger.info(f"每 {frames_to_skip} 幀擷取一次")

            frame_count = 0
            actual_frame_count = 0
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # 根據取樣率決定是否保存這一幀
                if frame_count % frames_to_skip == 0:
                    frame_path = self.frames_folder / f'frame_{actual_frame_count:04d}.jpg'
                    cv2.imwrite(str(frame_path), frame)
                    actual_frame_count += 1
                
                frame_count += 1

            self.logger.info(f"總幀數: {frame_count}, 擷取幀數: {actual_frame_count}")
            return frame_width, frame_height, actual_frame_count
        finally:
            cap.release()

    def run_yolo_detection(self):
        """執行 YOLO 偵測"""
        try:
            subprocess.run([
                "python", "detect.py",
                "--img", "640",
                "--conf", "0.1",
                "--device", "0",
                "--weights", str(self.weights_path),
                "--source", str(self.frames_folder)
            ], check=True)
        except subprocess.CalledProcessError as e:
            self.logger.error(f"YOLO 偵測失敗: {e}")
            raise
    ##找最新的exp資料夾
    def get_latest_exp_folder(self) -> Path:
        """
        找到最新的 exp 資料夾
        Returns:
            Path: 最新的 exp 資料夾路徑
        """
        base_dir = Path('runs/detect')
        if not base_dir.exists():
            raise RuntimeError(f"找不到 detect 資料夾: {base_dir}")

        # 取得所有 exp 開頭的資料夾
        exp_folders = [f for f in base_dir.iterdir() if f.is_dir() and f.name.startswith('exp')]
        
        if not exp_folders:
            raise RuntimeError("找不到任何 exp 資料夾")

        # 解析資料夾名稱中的數字
        def get_exp_number(folder_name: str) -> int:
            # 處理 'exp' 的情況
            if folder_name == 'exp':
                return 0
            # 處理 'expN' 的情況
            match = re.match(r'exp(\d+)', folder_name)
            return int(match.group(1)) if match else 0

        # 根據數字排序並取得最新的資料夾
        latest_exp = max(exp_folders, key=lambda x: get_exp_number(x.name))
        print(latest_exp)
        self.logger.info(f"找到最新的 exp 資料夾: {latest_exp}")
        return latest_exp
        
    def move_detected_images(self):
        """移動 YOLO 偵測後的圖片"""
        try:
            exp_folder = self.get_latest_exp_folder()
            if not exp_folder.exists():
                raise RuntimeError(f"找不到 YOLO 輸出資料夾: {exp_folder}")

            # 確保目標資料夾存在
            self.detected_folder.mkdir(parents=True, exist_ok=True)

            # 移動所有圖片檔案
            moved_count = 0
            for img_file in exp_folder.glob('*.jpg'):
                target_path = self.detected_folder / img_file.name
                shutil.move(str(img_file), str(target_path))
                moved_count += 1

            self.logger.info(f"成功移動 {moved_count} 個檔案到 {self.detected_folder}")

            if moved_count == 0:
                self.logger.warning("沒有找到任何圖片檔案需要移動")

        except Exception as e:
            self.logger.error(f"移動檔案時發生錯誤: {e}")
            raise

    def create_video(self, frame_width: int, frame_height: int):
        """將圖片合成為影片"""
        output_frames = sorted(os.listdir(self.detected_folder))
        if not output_frames:
            raise RuntimeError("沒有找到要處理的圖片")

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(str(self.output_video), fourcc, 
                            self.target_fps, (frame_width, frame_height))

        try:
            for frame_file in output_frames:
                frame_path = self.detected_folder / frame_file
                frame = cv2.imread(str(frame_path))
                
                if frame is None:
                    self.logger.warning(f"無法讀取圖片: {frame_path}")
                    continue
                    
                out.write(frame)
        finally:
            out.release()

    def process(self):
        """執行完整的處理流程"""
        try:
            self.logger.info("開始處理影片")
            self.setup_folders()
            
            self.logger.info("提取影片幀")
            frame_width, frame_height, frame_count = self.extract_frames()
            
            self.logger.info("執行 YOLO 偵測")
            self.run_yolo_detection()
            
            self.logger.info("移動偵測後的圖片")
            self.move_detected_images()
            
            self.logger.info("合成輸出影片")
            self.create_video(frame_width, frame_height)
            
            self.logger.info(f"處理完成，輸出影片: {self.output_video}")
            
        except Exception as e:
            self.logger.error(f"處理過程中發生錯誤: {e}")
            raise
    

if __name__ == "__main__":
    # 設定路徑
    VIDEO_PATH = 'C:/Users/Admin/Desktop/yolov9/yolov9/data/test/videos/nccdtest1.avi'
    FRAMES_FOLDER = 'C:/Users/Admin/Desktop/yolov9/yolov9/runs/result/TEST1/img/0215'
    DETECTED_FOLDER = 'C:/Users/Admin/Desktop/yolov9/yolov9/runs/result/TEST1/dimg/0215'
    OUTPUT_VIDEO = 'C:/Users/Admin/Desktop/yolov9/yolov9/runs/result/TEST1/vid/0215.avi'
    WEIGHTS_PATH = 'C:/Users/Admin/Desktop/yolov9/yolov9/runs/train/exp5/weights/best.pt'  

    # 設定每秒要擷取的畫面數量
    SAMPLE_RATE = 30  

    processor = VideoProcessor(
        video_path=VIDEO_PATH,
        frames_folder=FRAMES_FOLDER,
        detected_folder=DETECTED_FOLDER,
        output_video=OUTPUT_VIDEO,
        weights_path=WEIGHTS_PATH,
        sample_rate=SAMPLE_RATE  # 新增取樣率參數
    )
    
    processor.process()

INFO:__main__:開始處理影片
INFO:__main__:提取影片幀
INFO:__main__:來源影片 FPS: 29
INFO:__main__:取樣率: 每 0.03 秒一幀
INFO:__main__:每 1 幀擷取一次
INFO:__main__:總幀數: 387, 擷取幀數: 387
INFO:__main__:執行 YOLO 偵測
INFO:__main__:移動偵測後的圖片
INFO:__main__:找到最新的 exp 資料夾: runs\detect\exp39


runs\detect\exp39


INFO:__main__:成功移動 387 個檔案到 C:\Users\Admin\Desktop\yolov9\yolov9\runs\result\TEST1\dimg\0215
INFO:__main__:合成輸出影片
INFO:__main__:處理完成，輸出影片: C:\Users\Admin\Desktop\yolov9\yolov9\runs\result\TEST1\vid\0215.avi


## 單幀照片辨識


In [1]:
import torch
print(torch.__version__)  # PyTorch 版本
print(torch.cuda.is_available())  # 是否偵測到 GPU
print(torch.version.cuda)  # CUDA 版本


2.6.0+cu118
True
11.8


In [2]:
import os
HOME = os.getcwd()
print (HOME)

C:\Users\Admin\Desktop\yolov9


In [3]:
%cd {HOME}/yolov9

C:\Users\Admin\Desktop\yolov9\yolov9


In [10]:
import subprocess
import os
import json
import cv2
from pathlib import Path
import logging
import shutil
import time

def setup_logging():
    logging.basicConfig(level=logging.INFO)
    return logging.getLogger(__name__)

def run_yolo_detection(image_path: str, weights_path: str, output_folder: str):
    """執行 YOLO 偵測"""
    logger.info("開始 YOLO 偵測")
    output_dir = Path(output_folder)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    subprocess.run([
        "python", "detect.py",
        "--img", "640",
        "--conf", "0.1",
        "--device", "0",
        "--weights", weights_path,
        "--source", image_path ,
        "--save-txt"
    ], capture_output=True,check=True)
    
    logger.info("YOLO 偵測完成")

def get_latest_exp_folder() -> Path:
    """找出 YOLO 產生的最新 exp 資料夾"""
    base_dir = Path("runs/detect")
    if not base_dir.exists():
        raise RuntimeError(f"找不到 detect 資料夾: {base_dir}")
    
    exp_folders = [f for f in base_dir.iterdir() if f.is_dir() and f.name.startswith('exp')]
    if not exp_folders:
        raise RuntimeError("找不到任何 exp 資料夾")
    
    latest_exp = max(exp_folders, key=lambda x: int(x.name.replace("exp", "") or 0))
    logger.info(f"找到最新的 exp 資料夾: {latest_exp}")
    return latest_exp

def move_detected_image(exp_folder: Path, detected_folder: Path):
    """移動 YOLO 偵測後的圖片"""
    detected_folder.mkdir(parents=True, exist_ok=True)
    img_files = list(exp_folder.glob("*.jpg"))
    
    if not img_files:
        raise RuntimeError("未找到偵測後的圖片")
    
    target_img_path = detected_folder / img_files[0].name
    shutil.move(str(img_files[0]), str(target_img_path))
    logger.info(f"偵測後的圖片已儲存至: {target_img_path}")
    return target_img_path

def parse_detection_results(exp_folder: Path, output_json: str):
    """解析 YOLO 偵測結果並存成 JSON，並為每個粒子分配唯一編號"""

    # 1️確保 output_json 指向一個 JSON 檔案，而不是資料夾
    output_path = Path(output_json)
    if output_path.is_dir():  
        output_path = output_path / "detection_results.json"  # 自動加上 JSON 檔名
    elif not output_path.suffix:  
        output_path = output_path.with_suffix(".json")  # 如果沒有副檔名，補上 .json

    output_path.parent.mkdir(parents=True, exist_ok=True)  # 確保目錄存在

    txt_files = list(exp_folder.glob("labels/*.txt"))
    if not txt_files:
        raise RuntimeError("未找到偵測後的座標文件")

    detections = []
    with txt_files[0].open("r") as f:
        for i, line in enumerate(f.readlines()):
            values = line.strip().split()
            if len(values) < 5:
                continue
            
            class_id = int(values[0])
            x_center, y_center, width, height = map(float, values[1:5])

            detection = {
                "particle_id": i + 1,
                "class_id": class_id,
                "x_center": x_center,
                "y_center": y_center,
                "width": width,
                "height": height
            }
            detections.append(detection)

    # 2確保 JSON 檔案名稱正確後，再寫入
    with output_path.open("w", encoding="utf-8") as json_file:
        json.dump(detections, json_file, indent=4)

    print(f"偵測結果已儲存至: {output_path}")
    return str(output_path)

def process_image(image_path: str, weights_path: str, detected_folder: str, output_json: str):
    """完整處理流程: 影像辨識、結果儲存、座標輸出，並計算所需時間"""
    start_time = time.time()  # 記錄開始時間

    run_yolo_detection(image_path, weights_path, detected_folder)
    exp_folder = get_latest_exp_folder()
    detected_img = move_detected_image(exp_folder, Path(detected_folder))
    
    image_name = Path(image_path).stem  # 取得圖片名稱（不含副檔名）
    result_json = parse_detection_results(exp_folder, output_json)

    end_time = time.time()  # 記錄結束時間
    elapsed_time = end_time - start_time  # 計算總時間
    
    logger.info(f"處理完成，總耗時: {elapsed_time:.2f} 秒")
    return detected_img, result_json, elapsed_time

if __name__ == "__main__":
    logger = setup_logging()
    
    # 設定路徑
    IMAGE_PATH = "C:/Users/Admin/Desktop/yolov9/yolov9/data/test/images/20241123_006_jpg.rf.59efd44cb90bd453682ec62d69e5089a.jpg"
    WEIGHTS_PATH = "C:/Users/Admin/Desktop/yolov9/yolov9/runs/train/exp5/weights/best.pt"
    DETECTED_FOLDER = "C:/Users/Admin/Desktop/yolov9/yolov9/runs/result/TEST1/dimg/test2"
    OUTPUT_JSON = "C:/Users/Admin/Desktop/yolov9/yolov9/runs/result/TEST1/json/0305"
    
    process_image(IMAGE_PATH, WEIGHTS_PATH, DETECTED_FOLDER, OUTPUT_JSON)

INFO:__main__:開始 YOLO 偵測
INFO:__main__:YOLO 偵測完成
INFO:__main__:找到最新的 exp 資料夾: runs\detect\exp51
INFO:__main__:偵測後的圖片已儲存至: C:\Users\Admin\Desktop\yolov9\yolov9\runs\result\TEST1\dimg\test2\20241123_006_jpg.rf.59efd44cb90bd453682ec62d69e5089a.jpg
INFO:__main__:處理完成，總耗時: 8.41 秒


✅ 偵測結果已儲存至: C:\Users\Admin\Desktop\yolov9\yolov9\runs\result\TEST1\json\0305.json


In [6]:
import os

test_path = "C:/Users/Admin/Desktop/yolov9/yolov9/runs/result/TEST1/json/0305"
if not os.path.exists(test_path):
    os.makedirs(test_path, exist_ok=True)
print("資料夾可存取")


資料夾可存取
